# Extracting location names and data from Gibbon

In this note book we will put together and practice a lot of skills we have learned so far this term. Starting with just the raw text files from Gibbon's Decline and Fall we will create a DataFrame containing location names, location counts, and location data.

The code in this notebook may seem complex, but if you read through it carefully, you will likely understand what most of the code is doing.


## Set-up

In [2]:
# install necessary libraries. The "%%capture" stops the notebook from printing
# out all the insall output. Remove if you need to trouble shoot.
!pip install stanza

In [3]:
# install necessary libraries. The "%%capture" stops the notebook from printing
# out all the insall output. Remove if you need to trouble shoot.
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=470a3f0123b6045e15e3585aa050461c317b37775f043243efb341c6fa54da6c
  Stored in directory: /Users/Max/Library/Caches/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [20]:
# import necessary libraries
import os
import pandas as pd
import stanza
import json
import wget



## NLP pipeline
Now that all the necessary libraries have been installed and imported into our project, we need to set up our nlp pipeline. We will use [Stanza](https://stanfordnlp.github.io/stanza/).

In [27]:
# load stanza nlp pipeline that tokenizes and performs Named Entity Recognition
nlp_ner= stanza.Pipeline(lang='en', processors='tokenize, ner')

2023-11-05 17:20:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-05 17:20:32 INFO: Loading these models for language: en (English):
| Processor | Package          |
--------------------------------
| tokenize  | combined         |
| ner       | ontonotes_charlm |

2023-11-05 17:20:32 INFO: Using device: cpu
2023-11-05 17:20:32 INFO: Loading: tokenize
2023-11-05 17:20:32 INFO: Loading: ner
2023-11-05 17:20:33 INFO: Done loading processors!


## Load text data
If you are using a **Colab Notebook** you will need to run the cell below to get the text files.

Otherwise, you should have all of the text files for Gibbon's _Decline and Fall of the Roman Empire_ already downloaded from Canvas.

In [28]:
# load text files, Colab only
! git clone https://github.com/jdeen33/Gibbon_text.git

Cloning into 'Gibbon_text'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 71
Receiving objects: 100% (74/74), 4.20 MiB | 11.75 MiB/s, done.


## Extract location infromation from text file(s)

In [29]:
# create function that will take a text string as input and return a dictionary
# with locations and location counts from the text string
def get_locations_from_text(text):
    locations_dict = {}
    doc = nlp_ner(text)
    for sentence in doc.sentences:
        for token in sentence.tokens:
            if token.ner == 'S-GPE':
                if not token.text in locations_dict.keys():
                    locations_dict[token.text] = 1
                else:
                    locations_dict[token.text] += 1
            else:
                continue
    return locations_dict

You will need to choose which chapter you would like to extract locations from. For this example I will use Chapter 16.

For **Colab** it will look something like this:
`/content/Gibbon_text/gibbon_decline_volume1_chap16.txt`

For **Jupyter** it will look something like this:
`../text/gibbon_decline_and_fall/gibbon_decline_volume1_chap16.txt`

In [30]:
# identify the path to the text file you want to use
path_to_file = "text/gibbon_decline_and_fall/gibbon_decline_volume2_chap54.txt"

In [31]:
# read text from text file
with open(path_to_file, encoding='utf-8', mode='r') as f:
       text  = f.read()

In [32]:
# apply function to get locations and location counts
# this will take a few minutes
locations = get_locations_from_text(text)

In [33]:
# sanity check
locations

{'Armenia': 7,
 'Syria': 2,
 'Egypt': 2,
 'Rome': 6,
 'Nice': 2,
 'Samosata': 1,
 'Cappadocia': 2,
 'Pontus': 2,
 'Colonia': 2,
 'France': 4,
 'Siwas': 1,
 'Trebizond': 1,
 'Tephrice': 1,
 'Nicomedia': 1,
 'Ancyra': 1,
 'Ephesus': 2,
 'Constantinople': 2,
 'Thrace': 1,
 'Melitene': 1,
 'Philippopolis': 4,
 'Macedonia': 1,
 'Epirus': 1,
 'Bulgaria': 1,
 'Croatia': 1,
 'Dalmatia': 1,
 'Italy': 3,
 'Hungary': 1,
 'Jerusalem': 1,
 'Venice': 1,
 'Sicily': 1,
 'Milan': 1,
 'Orleans': 1,
 'Languedoc': 1,
 'Wickliff': 1,
 'England': 2,
 'Huss': 1,
 'Bohemia': 1,
 'Smithfield': 1,
 'Holland': 1}

In [115]:
# you may want to save the locations dictionary
path = './' # <-- Path of your choosing
file_name = 'locations_data.json'
with open(file_name, encoding='utf-8', mode='w') as f:
    json.dump(locations, f)

In [116]:
# convert dictionary to dataframe for easier processing
location_count_df = pd.DataFrame.from_dict(locations, orient='index').reset_index().rename(columns={'index':'place_name', 0:'count'})


In [117]:
# preview DataFrame
location_count_df.head()

,place_name,count
0,Armenia,7
1,Syria,2
2,Egypt,2
3,Rome,6
4,Nice,2


## Load data from Pleiades

In [118]:
# data from Pleiades, thanks to Peter Nadel!
if not os.path.isfile('places.csv'):  # checkin to see if we have this file or not
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/places.csv')
if not os.path.isfile('names.csv'):
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/names.csv')

In [119]:
# load and preview places DataFrame
places_df = pd.read_csv('places.csv')
places_df.head()

,created,description,details,provenance,title,uri,id,representative_latitude,representative_longitude,bounding_box_wkt
0,2010-06-24T14:11:06Z,"The ancient region known to the Romans as ""Gal...",<p>The Barrington Atlas Directory notes: FRA</p>,Barrington Atlas: BAtlas 1 D1 Gallia,Gallia,https://pleiades.stoa.org/places/993,993,46.705437,1.013706,"POLYGON ((8.672222 42.4395125, 8.672222 51.981..."
1,2020-01-10T20:52:00Z,"A Roman house in Pompeii (I, 6, 15), also know...",<p>The house was excavated in 1913 and 1914. T...,Pleiades,House of the Ceii,https://pleiades.stoa.org/places/999909607,999909607,40.750010,14.489506,"POLYGON ((14.4895058 40.7500099, 14.4895058 40..."
2,2015-05-28T11:48:45Z,"The so-called ""House of the Lararium of Achill...",NaN,Pleiades,House of the Lararium of Achilles,https://pleiades.stoa.org/places/999909608,999909608,40.750362,14.489286,"POLYGON ((14.489286 40.750362, 14.489286 40.75..."
3,2020-03-20T15:26:53Z,A necropolis with inhumations dating to the fi...,<p>A necropolis located close to Monte Bibele ...,Pleiades,Monte Tamburino necropolis,https://pleiades.stoa.org/places/999917206,999917206,44.272322,11.375880,"POLYGON ((11.3758803 44.2723222, 11.3758803 44..."
4,2018-05-30T03:08:22Z,The megalithic defensive circuit of Rusellae d...,NaN,Pleiades,Circuit wall of Rusellae,https://pleiades.stoa.org/places/999951524,999951524,42.829089,11.163588,"POLYGON ((11.1635884 42.8290888, 11.1635884 42..."


In [120]:
# load and preview names DataFrame
names_df = pd.read_csv('names.csv')
names_df.head()

,created,description,details,provenance,title,uri,id,place_id,name_type,language_tag,attested_form,romanized_form_1,romanized_form_2,romanized_form_3,association_certainty,transcription_accuracy,transcription_completeness,year_after_which,year_before_which
0,2010-06-24T14:11:06Z,NaN,NaN,Barrington Atlas: BAtlas 1 D1 Gallia,Gallia,https://pleiades.stoa.org/places/993/gallia,gallia,993,geographic,NaN,NaN,Gallia,NaN,NaN,certain,accurate,complete,NaN,NaN
1,2010-06-24T14:11:07Z,NaN,NaN,Barrington Atlas: BAtlas 1 D1 Gallia,Galli,https://pleiades.stoa.org/places/993/galli,galli,993,geographic,NaN,NaN,Galli,NaN,NaN,certain,accurate,complete,NaN,NaN
2,2015-06-23T17:19:22Z,NaN,NaN,Pleiades,House of the Ceii,https://pleiades.stoa.org/places/999909607/nam...,name.2015-06-23.9071239510,999909607,geographic,en,House of the Ceii,House of the Ceii,NaN,NaN,certain,accurate,complete,1700.0,2100.0
3,2017-06-26T23:54:15Z,NaN,NaN,Pleiades,Casa dei Ceii,https://pleiades.stoa.org/places/999909607/nam...,name.2017-06-26.2201864114,999909607,geographic,it,Casa dei Ceii,Casa dei Ceii,NaN,NaN,certain,accurate,complete,1700.0,2100.0
4,2020-12-26T17:59:50Z,NaN,NaN,Pleiades,House of L. Ceius Secundus,https://pleiades.stoa.org/places/999909607/hou...,house-of-l-ceius-secundus,999909607,geographic,en,House of L. Ceius Secundus,House of L. Ceius Secundus,NaN,NaN,certain,accurate,complete,NaN,NaN


In [121]:
# quick example: find 'Roma' in places DataFrame
places_df.loc[places_df['title'] == 'Roma']

,created,description,details,provenance,title,uri,id,representative_latitude,representative_longitude,bounding_box_wkt
21483,2018-06-07T19:48:13Z,The capital of the Roman Republic and Empire.,<p>The Barrington Atlas Directory notes: Roma/...,Barrington Atlas: BAtlas 43 B2 Roma,Roma,https://pleiades.stoa.org/places/423025,423025,41.891775,12.486137,"POLYGON ((12.486137 41.891775, 12.486137 41.89..."


In [122]:
# quick example: find 'Rome' in names DataFrame
names_df.loc[names_df['romanized_form_1'] == 'Rome']

,created,description,details,provenance,title,uri,id,place_id,name_type,language_tag,attested_form,romanized_form_1,romanized_form_2,romanized_form_3,association_certainty,transcription_accuracy,transcription_completeness,year_after_which,year_before_which
20810,2012-02-04T23:39:48Z,The modern English appellation,NaN,Barrington Atlas: BAtlas 43 B2 Roma,Rome,https://pleiades.stoa.org/places/423025/rome,rome,423025,geographic,en,Rome,Rome,NaN,NaN,certain,accurate,complete,1700.0,2100.0


## Extract data from Pleiades data
For each location in we identified from the text, we will extract extract the longitude, latitude, and a description. First we need to find each location in the Pleiades data.

In [123]:
def get_pleiades_id(location):
    """
    Iterates through all of the possible names in the names.csv file
    Returns None if no matched names
    """
    name_row = names_df.loc[names_df['attested_form'] == location]
    if len(name_row) == 1:
        return int(name_row.place_id.iloc[0])
    else:
        name_row = names_df.loc[names_df['romanized_form_1'] == location]
        if len(name_row) == 1:
            return int(name_row.place_id.iloc[0])
        else:
            name_row = names_df.loc[names_df['romanized_form_2'] == location]
            if len(name_row) == 1:
                return int(name_row.place_id.iloc[0])
            else:
                name_row = names_df.loc[names_df['romanized_form_3'] == location]
                if len(name_row) == 1:
                    return int(name_row.place_id.iloc[0])
                else:
                    return None

In [124]:
# apply the above founction to each row in our location count DataFrame and then
# add a new colum with the Pleiades id
location_count_df['pleiades_id'] = location_count_df['place_name'].apply(get_pleiades_id)

In [125]:
# preview new location count DataFrame.
# the NaN means we were unable to find the location in the Pleiades data.
location_count_df.head()

,place_name,count,pleiades_id
0,Armenia,7,NaN
1,Syria,2,NaN
2,Egypt,2,NaN
3,Rome,6,423025.0
4,Nice,2,NaN


In [126]:
# we can drop the rows with NaN values
location_count_df = location_count_df.dropna().reset_index(drop=True)

In [127]:
# preview updated location count DataFrame
location_count_df.head()

,place_name,count,pleiades_id
0,Rome,6,423025.0
1,Pontus,2,857287.0
2,Siwas,1,629075.0
3,Trebizond,1,857359.0
4,Nicomedia,1,511337.0


Now that we have a `pleiades_id` for each location from names.csv, we can use that information to get more data from the places.csv. It would be possible to combine the functions below into one, but I have seperated them out for clarity.

In [128]:
def get_description(pleiades_id):
    """return description from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.description.iloc[0]

In [129]:
def get_uri(pleiades_id):
    """return uri from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.uri.iloc[0]

In [130]:
def get_latitude(pleiades_id):
    """return latitude from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.representative_latitude.iloc[0]

In [131]:
# Challenge: Can you write a function to get the longitude data?
def get_longitude(pleiades_id):
    """return longitude from a pleiades id"""
    places_row = places_df.loc[places_df['id'] == pleiades_id]
    if len(places_row) == 1:
        return places_row.representative_longitude.iloc[0]




In [132]:
# add new column for description
location_count_df['description'] = location_count_df['pleiades_id'].apply(get_description)

In [133]:
# Challenge: can you write the code to add a colmn for the uri?
location_count_df['uri'] = location_count_df['pleiades_id'].apply(get_uri)

In [134]:
# add new column for latitude
location_count_df['latitude'] = location_count_df['pleiades_id'].apply(get_latitude)

In [135]:
# Challenge: can you write the code to add a colmn for the longitude?
location_count_df['longitude'] = location_count_df['pleiades_id'].apply(get_longitude)

Now that we have all the data we need, I am going to make a few little changes to the DataFrame.

In [136]:
# now that we have a uri we don't need the pleiades_id
location_count_df = location_count_df.drop(columns=['pleiades_id'])

In [137]:
# for our purposes we don't really need an index, so I will make the place_name column the index
location_count_df.set_index('place_name', inplace=True)

In [138]:
# final sanity check
location_count_df

,count,description,uri,latitude,longitude
place_name,,,,,
Rome,6,The capital of the Roman Republic and Empire.,https://pleiades.stoa.org/places/423025,41.891775,12.486137
Pontus,2,A region of northeastern Anatolia consisting o...,https://pleiades.stoa.org/places/857287,43.078685,34.742551
Siwas,1,"An ancient place, cited: BAtlas 64 E1 Sebastei...",https://pleiades.stoa.org/places/629075,39.749020,37.016607
Trebizond,1,An ancient settlement of Asia Minor on the sou...,https://pleiades.stoa.org/places/857359,41.004269,39.723312
Nicomedia,1,Nicomedia was founded in 712/11 BC as a Megari...,https://pleiades.stoa.org/places/511337,40.765191,29.919887
Ancyra,1,"An ancient place, cited: BAtlas 63 B1 Ancyra",https://pleiades.stoa.org/places/619103,39.943793,32.859333
Constantinople,2,"Capital city of the Eastern Roman Empire, the ...",https://pleiades.stoa.org/places/520998,41.006371,28.965352
Melitene,1,Melitene was an ancient city on the Tohma Rive...,https://pleiades.stoa.org/places/629040,38.382217,38.361152
Macedonia,1,The region of ancient Macedonia in the Balkan ...,https://pleiades.stoa.org/places/491656,41.250000,21.750000


## Save location data for further use

In [141]:
# create path and file name variables
path = "../code"# <-- set path variable (not necessary for Colab)
file_name = "Gibbon_Chapter_54_Locations" # <-- set file_name variable

In [145]:
# save DataFrame to a .csv file
location_count_df.to_csv(file_name) # <-- For Jupyter you may want to add path

OSError: Cannot save file into a non-existent directory: 'Gibbon_Chapter_54_Locations..'

In [ ]:
# Colab only
files.download(file_name)